In [ ]:
import importlib 

import pdal
import glob
import json
from rasterio_helpers import *
from proj_scope_vars import *
import rasterio
from rasterio.warp import reproject
import numpy as np

from arosics import DESHIFTER
from arosics import COREG_LOCAL
from geoarray import GeoArray
import os, shutil, glob
import numpy as np
import time
import re
from shapely.geometry import shape
import fiona

%load_ext autoreload
%aimport rasterio_helpers
%aimport proj_scope_vars
%autoreload 1
%autosave 300

In [ ]:
ndvi_out = f'{training_mask_dir}/{skysat_basename}_ndvi_training.tif'
skysat_ms_aligned_tif = glob.glob(f'{aligned_raster_dir}/{skysat_basename}*ms_aligned.tif')[0]
height_max_aligned_tif = glob.glob(f'{aligned_raster_dir}/*height_map_max*')[0]

with rasterio.open(skysat_ms_aligned_tif) as skysat_ms_aligned_ds:
    skysat_ms_aligned_np = skysat_ms_aligned_ds.read()
    skysat_ms_aligned_profile = skysat_ms_aligned_ds.profile
    skysat_ms_aligned_transform = skysat_ms_aligned_ds.transform
    skysat_ms_aligned_crs = skysat_ms_aligned_ds.crs
    

with rasterio.open(height_max_aligned_tif) as height_max_aligned_ds:
    height_max_aligned_np = height_max_aligned_ds.read()
    height_max_aligned_profile = height_max_aligned_ds.profile
    height_max_aligned_transform = height_max_aligned_ds.transform
    height_max_aligned_crs = height_max_aligned_ds.crs




In [ ]:
height_max_aligned_ds = rasterio.open(height_max_aligned_tif)
skysat_ms_aligned_ds = rasterio.open(skysat_ms_aligned_tif)

In [ ]:
height_max_aligned_ds.close()
skysat_ms_aligned_ds.close()

In [ ]:
print(skysat_ms_aligned_np.shape)
print(skysat_ms_aligned_profile)
print(skysat_ms_aligned_transform)
print(skysat_ms_aligned_crs)

print(height_max_aligned_np.shape)
print(height_max_aligned_profile)
print(height_max_aligned_transform)
print(height_max_aligned_crs)


In [ ]:
skysat_ms_aligned_tif = glob.glob(f'{aligned_raster_dir}/{skysat_basename}*ms_aligned.tif')[0]
print(skysat_ms_aligned_tif)

In [ ]:
#
skysat_ms_resampled_tif = skysat_ms_aligned_tif.replace('.tif', '_1m.tif' )

res = (1, 1)

with rasterio.open(skysat_ms_aligned_tif) as skysat_ms_aligned_ds:
    
    scale_factor = [skysat_ms_aligned_ds.res[i]/res[i] for i in range(2)]
    
    # resample data to target shape
    skysat_ms_aligned_np = skysat_ms_aligned_ds.read(
        out_shape=(
            skysat_ms_aligned_ds.count,
            int(skysat_ms_aligned_ds.height * scale_factor[0]),
            int(skysat_ms_aligned_ds.width  * scale_factor[1])
        ),
        resampling=Resampling.bilinear
    )
                      
    out_width  = skysat_ms_aligned_np.shape[-1]
    out_height = skysat_ms_aligned_np.shape[-2]

    # scale image transform
    transform = skysat_ms_aligned_ds.transform * skysat_ms_aligned_ds.transform.scale(
        (skysat_ms_aligned_ds.width / out_width),
        (skysat_ms_aligned_ds.height / out_height)
    )
    
    print(skysat_ms_aligned_np.shape)
    print('Transform before resample:\n', skysat_ms_aligned_ds.transform, '\n')
    print('Transform after resample:\n', transform)
    
        # Write outputs
    profile = skysat_ms_aligned_ds.profile
    print(profile)
    profile.update(
        {
            "transform": transform,
            "width": out_width,
            "height":out_height,
            "nodata": 0,  
        }
    )
    
    skysat_ms_aligned_masked_ds = mask_ds(get_ds(skysat_ms_aligned_np, profile), training_area_mask)
    
    skysat_ms_aligned_masked_np = skysat_ms_aligned_masked_ds.read()
    
    print(skysat_ms_aligned_masked_ds.profile)

#     with rasterio.open(skysat_ms_resampled_tif, "w", **profile) as dst:
#         # iterate through bands
#         for i in range(skysat_ms_aligned_np.shape[0]):
#               dst.write(skysat_ms_aligned_np[i].astype(rasterio.uint64), i+1)
    



In [ ]:
#Generate NDVI

skysat_ms_nir_np = skysat_ms_aligned_masked_ds.read(4).astype(rasterio.float64)
skysat_ms_red_np = skysat_ms_aligned_masked_ds.read(3).astype(rasterio.float64)
nodata=skysat_ms_aligned_masked_ds.nodata
profile=skysat_ms_aligned_masked_ds.profile
skysat_ndvi_tif = skysat_ms_aligned_tif.replace('.tif', '_1m_ndvi.tif' )

skysat_ms_ndvi_np = np.where(np.logical_or((skysat_ms_red_np != nodata), (skysat_ms_nir_np != nodata)), 
                            ((skysat_ms_nir_np-skysat_ms_red_np)/(skysat_ms_nir_np+skysat_ms_red_np)), 
                            -9999)
skysat_ms_ndvi_np = np.expand_dims(skysat_ms_ndvi_np, axis=0)
profile.update(
    {
        'dtype' : 'float64',
        'nodata': -9999,
        'count' : 1,
    }
)

skysat_ms_ndvi_ds = get_ds(skysat_ms_ndvi_np, profile)
save_ds(skysat_ms_ndvi_ds, skysat_ndvi_tif)
show(skysat_ms_ndvi_ds)

In [ ]:
a = [[1, 2], [3, 4]]
np.pad(a, ((0, 1), (0, 1)), 'constant', constant_values=(1))

In [ ]:
height_map_aligned_dl_binary_tif = f'{aligned_raster_dir}/ca_3dep_2018_sf_height_map_hag_binary.tif'

height_map_aligned_dl_tif = glob.glob(f'{aligned_raster_dir}/ca_3dep_2018_sf_height_map_hag*.tif')[0]
height_max_aligned_ds = rasterio.open(height_max_aligned_tif)
height_max_aligned_np=height_max_aligned_ds.read()
bin_np = np.where( height_max_aligned_np != height_max_aligned_ds.nodata, 1, -9999)
height_max_aligned_ds.close()
with rasterio.open(height_map_aligned_dl_binary_tif, "w", **profile) as ds:
    ds.write(bin_np)

In [ ]:
#Tree Mask
mask='/home/mike/geog_560_planet_proj/layers/ca_3dep_2018_sf_height_map_mask.gpkg'
skysat_ndvi_tif = glob.glob(f'{aligned_raster_dir}/{skysat_basename}*_1m_ndvi.tif')[0]
height_max_aligned_tif = glob.glob(f'{aligned_raster_dir}/ca_3dep_2018_sf_height_map_max*.tif')[0]
height_map_aligned_dl_tif = glob.glob(f'{aligned_raster_dir}/ca_3dep_2018_sf_height_map_hag*.tif')[0]

skysat_ms_ndvi_ds = rasterio.open(skysat_ndvi_tif)
height_max_aligned_ds = rasterio.open(height_max_aligned_tif)
height_dl_aligned_ds = rasterio.open(height_map_aligned_dl_tif)

height_dl_aligned_ds = mask_ds(height_dl_aligned_ds,training_area_mask)
height_max_aligned_ds = mask_ds(height_max_aligned_ds,training_area_mask)
skysat_ms_ndvi_ds = mask_ds(skysat_ms_ndvi_ds,training_area_mask)

profile = height_dl_aligned_ds.profile

print(height_dl_aligned_ds.profile)
print(height_max_aligned_ds.profile)
print(skysat_ms_ndvi_ds.profile)

height_max_aligned_np = height_max_aligned_ds.read()
height_dl_aligned_np  = height_dl_aligned_ds.read()
skysat_ms_ndvi_np     = skysat_ms_ndvi_ds.read()

height_dl_aligned_np=np.pad(height_dl_aligned_np, ((0,0), (0, 1), (0, 1)), 'constant', constant_values=(nodata))


In [ ]:
nodata = height_dl_aligned_ds.nodata



tree_criteria = np.logical_and(np.greater(skysat_ms_ndvi_np, 0.0), 
                             np.greater(height_max_aligned_np, 2.0))

ground_veg_criteria = np.logical_and(np.greater(skysat_ms_ndvi_np, 0.0), 
                       np.logical_and(np.less(height_dl_aligned_np, 2.0),
                                      np.greater(height_dl_aligned_np, 0.2)))

grass_criteria = np.logical_and(np.greater(skysat_ms_ndvi_np, 0.4), 
                               np.less(height_dl_aligned_np, 0.1))
                                   
street_criteria = np.logical_and(np.less(skysat_ms_ndvi_np, -0.3), 
                               np.less(height_dl_aligned_np, 0.2))
                                     
bldg_criteria = np.logical_and(np.less(skysat_ms_ndvi_np, -0.1), 
                               np.greater(height_dl_aligned_np, 3))

valid_data = np.logical_or( (skysat_ms_ndvi_np != skysat_ms_ndvi_ds.nodata),
                                (height_dl_aligned_np != height_dl_aligned_ds.nodata))
                                     
criterias = { "trees" : tree_criteria,
              "ground_veg" : ground_veg_criteria,
              "grass" : grass_criteria,
              "street" : street_criteria,
              "bldg" : bldg_criteria
            }

np_datas = {}
                                     
for label, criteria in criterias.items():   
    np_datas[label] = np.where( np.logical_and(valid_data,criteria), 1, -9999)
     
for label, np_data in np_datas.items():
    training_mask_tiff = f'{training_mask_dir}/{label}_training_mask.tif'
    
    with rasterio.open(training_mask_tiff, "w", **profile) as training_mask_ds:
        training_mask_ds.write(np_data)
        training_mask_ds.set_band_description(1, label)

In [ ]:
height_max_aligned_ds.profile

In [ ]:
height_dl_aligned_ds.profile

In [ ]:
a = [1, 2, 3, 4, 5]

In [ ]:
np.pad(a, (0,1), 'constant', constant_values=(1))